The question here is whether we can determine which patch a game is from. This has potential application for telling users that their playstyle is outdated, or for static stats by learning which features are determinate in making that decision.

In [1]:
%matplotlib inline
import numpy as np
from sklearn import svm, metrics
from sklearn import linear_model as lmod
from sklearn.cross_validation import train_test_split

In [2]:
patch = np.load('../datasets/np/champ_1_version_feature_10000.npy')
items = np.load('../datasets/np/champ_1_items_feature_10000.npy')
wins = np.load('../datasets/np/champ_1_stats.winner_feature_10000.npy').reshape((-1, 1))
features = np.hstack((items, wins))

The `patch` information is the output. It is trivially to reformat into the customary -1, 1

In [6]:
p511 = patch == '5.11'
output = np.ones(patch.shape)
output[p511] = -1

In [4]:
FS = lmod.RandomizedLogisticRegression()
FS.fit(features, output)

RandomizedLogisticRegression(C=1, fit_intercept=True,
               memory=Memory(cachedir=None), n_jobs=1, n_resampling=200,
               normalize=True, pre_dispatch='3*n_jobs', random_state=None,
               sample_fraction=0.75, scaling=0.5, selection_threshold=0.25,
               tol=0.001, verbose=False)

In [7]:
FS.get_support().nonzero()

(array([ 84, 223]),)

So on a single-champion basis (with an admittedly small dataset size, 255 samples total) we are getting no features selected. Dass ist nicht so gut.

**EDIT** After a second run with an order of a magnitude more data, RLR extracts two features of import.

In [8]:
M = svm.LinearSVC()
train_feat, test_feat, train_out, test_out = train_test_split(features, output, test_size=0.33)
M.fit(train_feat, train_out)
pred = M.predict(test_feat)
fpr, tpr, _ = metrics.roc_curve(test_out, pred)
auc = metrics.auc(fpr, tpr)
print(auc)

0.501591735967


Only slightly better than random. Let's see how things look using only the RLR features.

In [9]:
support = FS.get_support()
M = svm.LinearSVC()
M.fit(train_feat[:, support], train_out)
pred = M.predict(test_feat[:, support])
fpr, tpr, _ = metrics.roc_curve(test_out, pred)
auc = metrics.auc(fpr, tpr)
print(auc)

0.536609927235


No significant improvement (other than in training time). The only possibility that I can think of is that Annie builds did not change much between the two patches. I need to re-learn statistical significance testing and confidence intervals to confirm.